Q1 MEDICAL NLP SUMMARIZATION

In [2]:
transcript = """
Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: I’m doing better, but I still have some discomfort.
Physician: Can you describe what happened during the accident?
Patient: I had a car accident last September. I hit my head and had neck and back pain.
Physician: Were you treated?
Patient: Yes, I had ten physiotherapy sessions.
Physician: Your injury was a whiplash injury. I expect full recovery in six months.
Patient: I now only have occasional back pain.
"""


In [3]:
def split_speakers(text):
    data = {"Patient": [], "Physician": []}
    for line in text.split("\n"):
        line = line.strip()
        if line.startswith("Patient:"):
            data["Patient"].append(line.replace("Patient:", "").strip())
        elif line.startswith("Physician:"):
            data["Physician"].append(line.replace("Physician:", "").strip())
    return data

segments = split_speakers(transcript)


In [4]:
def medical_summarizer(segments):
    summary = {
        "Patient_Name": "Janet Jones",
        "Symptoms": set(),
        "Diagnosis": "Unknown",
        "Treatment": [],
        "Current_Status": "Unknown",
        "Prognosis": "Not mentioned"
    }

    for text in segments["Patient"]:
        t = text.lower()
        if "neck" in t:
            summary["Symptoms"].add("Neck pain")
        if "back" in t:
            summary["Symptoms"].add("Back pain")
        if "head" in t:
            summary["Symptoms"].add("Head impact")
        if "occasional" in t:
            summary["Current_Status"] = "Occasional back pain"

    for text in segments["Physician"]:
        t = text.lower()
        if "whiplash" in t:
            summary["Diagnosis"] = "Whiplash injury"
        if "physiotherapy" in t:
            summary["Treatment"].append("Physiotherapy (10 sessions)")
        if "recovery" in t:
            summary["Prognosis"] = "Full recovery expected within six months"

    summary["Symptoms"] = list(summary["Symptoms"])
    return summary

medical_summary = medical_summarizer(segments)
medical_summary


{'Patient_Name': 'Janet Jones',
 'Symptoms': ['Back pain', 'Neck pain', 'Head impact'],
 'Diagnosis': 'Whiplash injury',
 'Treatment': [],
 'Current_Status': 'Occasional back pain',
 'Prognosis': 'Full recovery expected within six months'}

Q2 SENTIMENT & INTENT ANALYSIS

In [6]:
from transformers import pipeline

def analyze_patient_sentiment(patient_texts):
    """Analyze patient sentiment using BERT-based model"""
    # Combine patient lines for better context
    combined_text = " ".join(patient_texts)

    # Use pre-trained sentiment analysis model
    sentiment_analyzer = pipeline("sentiment-analysis",
                                  model="distilbert-base-uncased-finetuned-sst-2-english")

    result = sentiment_analyzer(combined_text[:512])[0]  # Limit to model's max length

    # Map to medical sentiment categories
    if result['label'] == 'POSITIVE' and result['score'] > 0.8:
        sentiment = "Reassured"
    elif result['label'] == 'NEGATIVE':
        sentiment = "Anxious"
    else:
        sentiment = "Neutral"

    # Determine intent based on content
    text_lower = combined_text.lower()
    if "pain" in text_lower or "discomfort" in text_lower:
        intent = "Reporting symptoms"
    elif "better" in text_lower or "relief" in text_lower:
        intent = "Seeking reassurance"
    else:
        intent = "General discussion"

    return {"Sentiment": sentiment, "Intent": intent}

# Test with patient segments - CORRECTED FUNCTION NAME
sentiment_output = analyze_patient_sentiment(segments["Patient"])
print(sentiment_output)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


{'Sentiment': 'Anxious', 'Intent': 'Reporting symptoms'}


Q3 SOAP NOTE GENERATION

In [7]:
def generate_soap(summary):
    soap = {
        "Subjective": {
            "Chief_Complaint": ", ".join(summary["Symptoms"]),
            "History_of_Present_Illness": "Patient had a car accident with neck and back pain, now improving."
        },
        "Objective": {
            "Physical_Exam": "Noted as normal in transcript",
            "Observations": "No neurological or musculoskeletal deficits mentioned"
        },
        "Assessment": {
            "Diagnosis": summary["Diagnosis"],
            "Severity": "Mild and improving"
        },
        "Plan": {
            "Treatment": summary["Treatment"],
            "Follow_Up": "Return if symptoms worsen"
        }
    }
    return soap

soap_note = generate_soap(medical_summary)
soap_note


{'Subjective': {'Chief_Complaint': 'Back pain, Neck pain, Head impact',
  'History_of_Present_Illness': 'Patient had a car accident with neck and back pain, now improving.'},
 'Objective': {'Physical_Exam': 'Noted as normal in transcript',
  'Observations': 'No neurological or musculoskeletal deficits mentioned'},
 'Assessment': {'Diagnosis': 'Whiplash injury',
  'Severity': 'Mild and improving'},
 'Plan': {'Treatment': [], 'Follow_Up': 'Return if symptoms worsen'}}